# Speeding up CDF in Python with Numba

Saeed Amen - https://www.cuemacro.com - saeed@cuemacro.com

In this notebook, we'll focus on speeding up the normal Cumulative Distribution Function in Python. This function is used often, for example in the Black-Scholes formula for pricing options. The normal CDF tends to be one of the slower arithmetic operations in the pricing.

If we are only pricing a single option this probably doesn't matter. However, if we are using it in a loop (eg. for pricing many options, in a numerical solver etc.), speeding up this bottleneck could be very helpful indeed. We'll go through a few different variations of the CDF in Python to see how can speed it up, in particular using Numba. We'll be running benchmarks using `timeit` (obviously these will give different timings on other machines

I've been using the FinancePy library a lot which can price options available from https://github.com/domokane/FinancePy (and made a few very small contributions to speeding up parts). We'll also be benchmarking the CDF function implemented in FinancePy later too.

**If you are interested in speeding up your Python code, I run a workshop, which I can teach at your firm on "Python, alternative data, natural language processing and large datasets", part of which is about using Numba and other tools to speed up numerical calculations.**

## What is Numba (http://numba.pydata.org/)?

Numba understands many `numpy` functions and can speed them up, and it understands NumPy data structures like `narray` objects. It doesn't understand Pandas DataFrames, or indeed functions from scipy. Hence, when you need to use Numba with Pandas, you will need to rewrite you code to expose the underlying `narray` objects used to store the data in the DataFrame. On an experimental basis, Numba does support classes, but in general, I think it's easier to use with functions. 

Basically, Numba compiles the code at runtime using an LLVM, and this compilation can also be cached. I have tended to find it easier to use than alternatives like Cython and your code is still quite recognisable as "Python" (albeit with various additional decorators). One of the big no-nos in Python, is to use lots of tightly nested for loops. Typically, you should try to vectorize your calculations to get speedups with NumPy. However, 

You will also have to decorate your Python code to indicate to the LLVM that you want it to be compiled by Numba (or jitted). Ideally, your function will be speeded up most if there is no need to interact with the Python interpreter when running it. In these cases, you can try decorating your code with the `njit` keyword. Numba, will tell you if needs to interact with the Python interpreter by throwing an error (in which case you will need to use the `jit` keyword)! In some cases, you may also need to specify the types of the inputs and outputs in the decorator. There are all sorts of other flags, such as 
* `fastmath` (speeds up maths calculations but might be accurate to fewer decimal places) 
* `cache` (which will cache the compiled code).
* `vectorize` that can be used to run the same computation across many elements of a vector
* `parallel` that can be used to parallelize for loops
    
Numba lets you target the CPU or GPU. For GPU, you'll need to rewrite your code more, usually in a CUDA-like way, and you'll need to understand how a GPU works properly.

## Using scipy

Typically, when calculating the normal CDF, one of the most common ways is to use scipy, an in particular to run `scipy.stats.norm.cdf`. For our use case we shall assume that our input `n` is always `0.5`

In [50]:
from scipy.stats import norm

n = 0.5

### Benchmarking `scipy.stats.norm.cdf`

If we run a benchmark of this function it takes around 60 µs on my machine when using `timeit`, which runs it many times.

In [40]:
%%timeit

norm.cdf(n)

63.2 µs ± 858 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Benchmarking `scipy.special.ndtr`

If we explore the code for scipy on GitHub, given it's all open source, we find that underneath, it actually calls the `scipy.special.ndtr` function. Does calling this function directly help in terms of speeding up our calculation? Amazingly, simply calling this lower level function makes our code run in around 600ns, which is a massive speedup! Simply by doing a lower level call, we have speeded up our calculation by 100x!

This isn't a unique situation. Very often for example, when using Pandas, underneath a lot of the arithmetic operations are done using NumPy. By calling the NumPy methods directly we can speed up our calculation in many instances.

In [49]:
from scipy.special import ndtr

In [41]:
%%timeit

ndtr(n)

629 ns ± 5.06 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Rewriting CDF (with and without Numba)

### Benchmarking `scipy.special.ndtr` converted into Python

If we go through the source code of ndtr, we find that underneath it is actually C code, and these can be called from Cython code (see https://docs.scipy.org/doc/scipy/reference/special.cython_special.html). What if we try to rewrite this code as Python, and try to call it? We'll have to take out a bunch of curly brackets and make the code Pythonic. Thankfully it's sufficiently simple code that it isn't too difficult. There are Python equivalents for `fabs`, `erf` and `erfc` in the Python `math` module, which we can use after importing (note, `erf` and `erfc` are actually also included in the below C code, so we could have converted these if we'd wanted to).

The C code can be found at https://github.com/scipy/scipy/blob/master/scipy/special/cephes/ndtr.c

```
double ndtr(double a)
{
    double x, y, z;

    if (cephes_isnan(a)) {
	sf_error("ndtr", SF_ERROR_DOMAIN, NULL);
	return (NPY_NAN);
    }

    x = a * NPY_SQRT1_2;
    z = fabs(x);

    if (z < NPY_SQRT1_2)
	y = 0.5 + 0.5 * erf(x);

    else {
	y = 0.5 * erfc(z);

	if (x > 0)
	    y = 1.0 - y;
    }

    return (y);
}
```

In [98]:
import numpy as np

from math import fabs, erf, erfc, exp

NPY_SQRT1_2 = 1.0/ np.sqrt(2)

In [87]:
def ndtr_python(a):

    if (np.isnan(a)):
        return np.nan

    x = a * NPY_SQRT1_2;
    z = fabs(x)

    if (z < NPY_SQRT1_2):
        y = 0.5 + 0.5 * erf(x)

    else:
        y = 0.5 * erfc(z)

        if (x > 0):
            y = 1.0 - y

    return y

When we benchmark it we find it's slower, at around 1.4 µs. This probably isn't surprising given that `ndtr` was calling lower level C code underneath.

In [88]:
%%timeit

ndtr_python(n)

1.39 µs ± 14.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


We can create a Numba-ized version of `ndtr`, because the code is sufficiently simple, by adding the `njit` decorator at the top.

In [89]:
from numba import njit

In [99]:
@njit(cache=True, fastmath=True)
def ndtr_numba(a):

    if (np.isnan(a)):
        return np.nan

    x = a * NPY_SQRT1_2;
    z = fabs(x)

    if (z < NPY_SQRT1_2):
        y = 0.5 + 0.5 * erf(x)

    else:
        y = 0.5 * erfc(z)

        if (x > 0):
            y = 1.0 - y

    return y

Using Numba yields a time of around 170ns, much quicker, than the pure Python implementation.

In [100]:
%%timeit

ndtr_numba(n)

171 ns ± 0.478 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Comparing with FinancePy's CDF

In the above cases, we were using the same algorithm for computing the CDF, just executing it in different ways. Here, we shall benchmark the performance of the CDF algorithm in FinancePy which is different. It is a fast approximation of the CDF good to around 6 decimal places. 

### Benchmarking FinancePy's CDF (with and without Numba)

If we benchmark FinancePy's CDF function `N(.)` (see https://github.com/domokane/FinancePy/blob/master/financepy/finutils/FinMath.py). We shall show the performance without the code being Numba-ized (or jitted) and with the jitted code actually used in the FinancePy library.

In [97]:
# Based on code by Dominic O'Kane
# Copyright (C) 2018, 2019, 2020 Dominic O'Kane

def N_python(x):
    ''' Fast Normal CDF function based on Hull OFAODS  4th Edition Page 252. 
    This function is accurate to 6 decimal places. '''

    a1 = 0.319381530
    a2 = -0.356563782
    a3 = 1.781477937
    a4 = -1.821255978
    a5 = 1.330274429
    g = 0.2316419

    k = 1.0 / (1.0 + g * fabs(x))
    k2 = k * k
    k3 = k2 * k
    k4 = k3 * k
    k5 = k4 * k

    if x >= 0.0:
        c = (a1 * k + a2 * k2 + a3 * k3 + a4 * k4 + a5 * k5)
        phi = 1.0 - c * exp(-x*x/2.0) * NPY_SQRT1_2
    else:
        phi = 1.0 - N(-x)

    return phi

The pure Python function `N_python` runs in around 720 ns, quicker than `ndtr_python` but slower than the jitted `ndtr_numba`.

In [93]:
%%timeit

N_python(n)

722 ns ± 5.31 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Let's now import the jitted `N(.)` function which is identical to the above, but it also has the `njit` declaration at the top. We find it is even faster than our jitted `ndtr_numba` function at close to 150ns. 

In [94]:
from financepy.finutils.FinMath import *

In [95]:
%%timeit

N(n)

148 ns ± 0.929 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Final check

Just as a final check, let's see if all the outputs are similar! They are, to 8 decimal places. 

In [96]:
print(norm.cdf(n))
print(ndtr(n))
print(ndtr_python(n))
print(ndtr_numba(n))
print(N_python(n))
print(N(n))

0.6914624612740131
0.6914624612740131
0.691462461274013
0.6914624612740131
0.6914624677873249
0.6914624677873249


## Conclusion

We have seen that it is possible to speed up the CDF considerably, by using lower level calls in SciPy, and later translating the code into Python and then jitting it. We also compared the speed to FinancePy (both pure Python and jitted), which uses a different CDF algorithm.

Here are some generalized steps that I would recommend to use when you find a bottleneck in your Python based numerical code:

* Try to use a lower level function to access the same calculation
* Translate the lower level function (if it's in C) to Python
* More broadly if we want to jit the code, we:
    * may use NumPy functions (but can't use Pandas for example)
    * need to avoid more complicated Python structures/calls (so can use `njit` flag)
* Once you are happy with your "simplified" Python code, we then need to jit it
* Apply Numba decorator to jit the code
    * with types if necessary
    * add any additional flags
* It is easier to debug non-jitted code, so you might need to add/remove the Numba decorator when debugging

Should you try to jit all code? Probably not, because it can take time to rewrite (depending on how complicated you code is, and how much abstraction there is in your code). In many cases, it won't be necessary to jit your code, but where a particular part of code is a bottleneck, it might be worth spending the time to do rewrite it in Numba/jit it.

Is this stuff *easy*, no not really, but once you get experience in doing this sort of thing, you can know what to look for when speeding up your Python code.